# Coleta e Limpeza do Dataset VALE3.SA para LSTM
Este notebook baixa os dados com yFinance, salva o bruto e gera um arquivo limpo contendo apenas `Date` e `Close`, ordenado por data.

In [2]:
# Imports necessários
import yfinance as yf
import pandas as pd
import os

In [3]:
# Definições de parâmetros
ticker = 'VALE3.SA'
start_date = '2018-01-01'
end_date = '2024-07-20'

# Caminhos
notebook_dir = os.getcwd()
base_dir = os.path.abspath(os.path.join(notebook_dir, '..', 'data'))
os.makedirs(base_dir, exist_ok=True)

# Segurança ao deletar: remove só arquivos .csv
for filename in os.listdir(base_dir):
    file_path = os.path.join(base_dir, filename)
    if os.path.isfile(file_path) and filename.endswith('.csv'):
        os.remove(file_path)

raw_file = os.path.join(base_dir, f'{ticker}.csv')
clean_file = os.path.join(base_dir, 'dataset.csv')

# (Opcional) Debug
print(f"Salvará arquivos em: {base_dir}")


Salvará arquivos em: j:\Projetos\FASE4-ML\data


In [4]:
# Download e salvamento
print(f"Baixando dados de {ticker}...")
df = yf.download(ticker, start=start_date, end=end_date)

# Verificação de sucesso
if df.empty:
    raise ValueError(f"Nenhum dado foi baixado para o ticker {ticker}.")

df.reset_index(inplace=True)

if 'Date' not in df.columns:
    raise RuntimeError("A coluna 'Date' não foi encontrada após reset_index.")

df.to_csv(raw_file, index=False)
print(f"Arquivo bruto salvo em: {raw_file}")
print(df.head())


Baixando dados de VALE3.SA...
YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed

Arquivo bruto salvo em: j:\Projetos\FASE4-ML\data\VALE3.SA.csv
Price        Date      Close       High        Low       Open    Volume
Ticker              VALE3.SA   VALE3.SA   VALE3.SA   VALE3.SA  VALE3.SA
0      2018-01-02  23.603912  23.615228  22.879726  22.879726  14156500
1      2018-01-03  23.462471  23.694437  23.366289  23.666148  12744200
2      2018-01-04  23.558649  23.971661  23.490757  23.654831  18433000
3      2018-01-05  23.926401  23.926401  23.371947  23.519046  15251300
4      2018-01-08  24.458227  24.458227  23.988639  23.988639  14542800


In [5]:
df = pd.read_csv(raw_file)

# Seleciona colunas e limpa datas inválidas
df = df[['Date', 'Close']]
df = df[pd.to_datetime(df['Date'], errors='coerce').notna()]
df['Date'] = pd.to_datetime(df['Date'])

# Remove linhas com Close vazio
df = df[df['Close'].notna()]

# Ordena e reseta índice
df = df.sort_values('Date').reset_index(drop=True)

# Verificação final
if df.empty:
    raise ValueError("O DataFrame final está vazio após limpeza.")

# Salva e remove bruto
df.to_csv(clean_file, index=False)
print(f"Arquivo limpo salvo em: {clean_file}")
os.remove(raw_file)


Arquivo limpo salvo em: j:\Projetos\FASE4-ML\data\dataset.csv
